## Gradient Boost Regression

- ### Problem Statement
    - This dataset comprises used cars sold on cardehko.com in India as well as important features of these cars.
    - If user can predict the price of the car based on input features.
    - Prediction results can be used to give new seller the price suggestion based on market condition.

- ### Data Collection
    - The Dataset is collected from scrapping the cardekho.com website
    - The data consists of 13 columns and 15411 rows.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

In [2]:
df = pd.read_csv('Cardekho Imputed Data.csv', index_col=[0])

In [3]:
df.head()

,car_name,brand,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,Maruti Alto,Maruti,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,Hyundai Grand,Hyundai,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,Hyundai i20,Hyundai,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,Maruti Alto,Maruti,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,Ford Ecosport,Ford,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


### Data Cleaning

In [4]:
df.isnull().sum()

car_name             0
brand                0
model                0
vehicle_age          0
km_driven            0
seller_type          0
fuel_type            0
transmission_type    0
mileage              0
engine               0
max_power            0
seats                0
selling_price        0
dtype: int64

In [5]:
df = df.drop(columns=['car_name', 'brand'], axis=1)

In [6]:
df.head()

,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


In [7]:
df['model'].unique()

array(['Alto', 'Grand', 'i20', 'Ecosport', 'Wagon R', 'i10', 'Venue',
       'Swift', 'Verna', 'Duster', 'Cooper', 'Ciaz', 'C-Class', 'Innova',
       'Baleno', 'Swift Dzire', 'Vento', 'Creta', 'City', 'Bolero',
       'Fortuner', 'KWID', 'Amaze', 'Santro', 'XUV500', 'KUV100', 'Ignis',
       'RediGO', 'Scorpio', 'Marazzo', 'Aspire', 'Figo', 'Vitara',
       'Tiago', 'Polo', 'Seltos', 'Celerio', 'GO', '5', 'CR-V',
       'Endeavour', 'KUV', 'Jazz', '3', 'A4', 'Tigor', 'Ertiga', 'Safari',
       'Thar', 'Hexa', 'Rover', 'Eeco', 'A6', 'E-Class', 'Q7', 'Z4', '6',
       'XF', 'X5', 'Hector', 'Civic', 'D-Max', 'Cayenne', 'X1', 'Rapid',
       'Freestyle', 'Superb', 'Nexon', 'XUV300', 'Dzire VXI', 'S90',
       'WR-V', 'XL6', 'Triber', 'ES', 'Wrangler', 'Camry', 'Elantra',
       'Yaris', 'GL-Class', '7', 'S-Presso', 'Dzire LXI', 'Aura', 'XC',
       'Ghibli', 'Continental', 'CR', 'Kicks', 'S-Class', 'Tucson',
       'Harrier', 'X3', 'Octavia', 'Compass', 'CLS', 'redi-GO', 'Glanza',
       

In [8]:
num_features = df.select_dtypes(exclude="object").columns
num_features

Index(['vehicle_age', 'km_driven', 'mileage', 'engine', 'max_power', 'seats',
       'selling_price'],
      dtype='object')

In [9]:
cat_features = df.select_dtypes(include="object").columns
cat_features

Index(['model', 'seller_type', 'fuel_type', 'transmission_type'], dtype='object')

In [10]:
discrete_features = [feature for feature in num_features if len(df[feature].unique()) < 25]
discrete_features

['vehicle_age', 'seats']

In [11]:
continuous_features = [feature for feature in num_features if feature not in discrete_features]
continuous_features

['km_driven', 'mileage', 'engine', 'max_power', 'selling_price']

In [12]:
## Independent and Dependent Features
X = df.drop(['selling_price'], axis=1)
y = df['selling_price']

In [13]:
X.head()

,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats
0,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5
1,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5
2,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5
3,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5
4,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5


In [14]:
y

0         120000
1         550000
2         215000
3         226000
4         570000
          ...   
19537     250000
19540     925000
19541     425000
19542    1225000
19543    1200000
Name: selling_price, Length: 15411, dtype: int64

### Feature Encoding and Scaling

In [15]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [16]:
X['model'] = le.fit_transform(X['model'])

In [17]:
X.head()

,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats
0,7,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5
1,54,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5
2,118,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5
3,7,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5
4,38,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5


In [18]:
len(df['seller_type'].unique()), len(df['fuel_type'].unique()), len(df['transmission_type'].unique())

(3, 5, 2)

In [19]:
df['fuel_type'].unique()

array(['Petrol', 'Diesel', 'CNG', 'LPG', 'Electric'], dtype=object)

In [20]:
num_features = X.select_dtypes(exclude="object").columns
ohe_columns = ['seller_type', 'fuel_type', 'transmission_type']

In [21]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
## In any ensemble technique, we don't need to scale the data -> It is just for practice
from sklearn.compose import ColumnTransformer

In [22]:
numeric_transformer = StandardScaler()
ohe_transformer = OneHotEncoder(drop='first')

In [23]:
preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", ohe_transformer, ohe_columns),
        ("StandardScaler", numeric_transformer, num_features)
    ], remainder='passthrough'
)

In [24]:
X = preprocessor.fit_transform(X)

In [25]:
pd.DataFrame(X)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,-1.519714,0.983562,1.247335,-0.000276,-1.324259,-1.263352,-0.403022
1,1.0,0.0,0.0,0.0,0.0,1.0,1.0,-0.225693,-0.343933,-0.690016,-0.192071,-0.554718,-0.432571,-0.403022
2,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.536377,1.647309,0.084924,-0.647583,-0.554718,-0.479113,-0.403022
3,1.0,0.0,0.0,0.0,0.0,1.0,1.0,-1.519714,0.983562,-0.360667,0.292211,-0.936610,-0.779312,-0.403022
4,0.0,0.0,1.0,0.0,0.0,0.0,1.0,-0.666211,-0.012060,-0.496281,0.735736,0.022918,-0.046502,-0.403022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15406,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.508844,0.983562,-0.869744,0.026096,-0.767733,-0.757204,-0.403022
15407,0.0,0.0,0.0,0.0,0.0,1.0,1.0,-0.556082,-1.339555,-0.728763,-0.527711,-0.216964,-0.220803,2.073444
15408,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.407551,-0.012060,0.220539,0.344954,0.022918,0.068225,-0.403022
15409,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.426247,-0.343933,72.541850,-0.887326,1.329794,0.917158,2.073444


In [26]:
## Train Test Split
from sklearn.model_selection import train_test_split

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

### Model Training and Model Selection

In [28]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [29]:
## Generic Function for Model Evaluation
def model_evaluation(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, mse, rmse, r2_square

In [30]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "Gradient Boost Regressor": GradientBoostingRegressor()
   
}

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_mse, model_train_rmse, model_train_r2 = model_evaluation(y_train, y_train_pred)

    model_test_mae , model_test_mse, model_test_rmse, model_test_r2 = model_evaluation(y_test, y_test_pred)

    
    print(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- Mean Squared Error: {:.4f}".format(model_train_mse))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- Mean Squared Error: {:.4f}".format(model_test_mse))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    
    print('='*35)
    print('\n')

Linear Regression
Model performance for Training set
- Root Mean Squared Error: 559313.7144
- Mean Absolute Error: 268437.6549
- Mean Squared Error: 312831831093.0353
- R2 Score: 0.6183
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 507556.7988
- Mean Absolute Error: 281329.9042
- Mean Squared Error: 257613903997.1092
- R2 Score: 0.6575


Lasso
Model performance for Training set
- Root Mean Squared Error: 559313.7247
- Mean Absolute Error: 268436.5960
- Mean Squared Error: 312831842666.6471
- R2 Score: 0.6183
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 507555.9012
- Mean Absolute Error: 281329.3313
- Mean Squared Error: 257612992800.0465
- R2 Score: 0.6575


Ridge
Model performance for Training set
- Root Mean Squared Error: 559314.5337
- Mean Absolute Error: 268393.2589
- Mean Squared Error: 312832747644.5701
- R2 Score: 0.6183
----------------------------------
Model performance for Test se

### Hyperparameter Tuning

In [31]:
gb_params = {
    'loss': ['squared_error', 'absolute_error', 'huber', 'quantile'],
    'learning_rate': [0.001, 0.01, 0.1, 0.2, 0.3],
    'n_estimators': [100, 200, 300, 400, 500],
    'subsample': [0.5, 0.7, 0.9, 1.0],
    'criterion': ['friedman_mse', 'squared_error'],
    'min_samples_split': [2, 5, 10],
    'max_depth': [3, 5, 10, 15, 20],
    'max_features': [None, 'sqrt', 'log2']
}

In [32]:
randomcv_models = [("Gradient Boost Regressor", GradientBoostingRegressor(), gb_params)]

In [33]:
from sklearn.model_selection import RandomizedSearchCV

model_param = {}
for name, model, params in randomcv_models:
    random = RandomizedSearchCV(estimator=model, param_distributions=params, n_iter=100, cv=5, verbose=2, n_jobs=-1)
    random.fit(X_train, y_train)
    model_param[name] = random.best_params_

for model_name in model_param:
    print(f"---------------- Best Params for {model_name} -------------------")
    print(model_param[model_name])

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV] END criterion=friedman_mse, learning_rate=0.2, loss=huber, max_depth=3, max_features=sqrt, min_samples_split=5, n_estimators=400, subsample=0.9; total time=   1.7s
[CV] END criterion=friedman_mse, learning_rate=0.2, loss=quantile, max_depth=3, max_features=None, min_samples_split=10, n_estimators=500, subsample=0.7; total time=   3.2s
[CV] END criterion=friedman_mse, learning_rate=0.2, loss=quantile, max_depth=3, max_features=None, min_samples_split=10, n_estimators=500, subsample=0.7; total time=   3.2s
[CV] END criterion=friedman_mse, learning_rate=0.2, loss=quantile, max_depth=3, max_features=None, min_samples_split=10, n_estimators=500, subsample=0.7; total time=   3.2s
[CV] END criterion=friedman_mse, learning_rate=0.2, loss=quantile, max_depth=3, max_features=None, min_samples_split=10, n_estimators=500, subsample=0.7; total time=   3.1s
[CV] END criterion=friedman_mse, learning_rate=0.2, loss=quantile, max_depth

/Users/dhruvsmac/Desktop/development/machine learning/.venv/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV] END criterion=friedman_mse, learning_rate=0.01, loss=squared_error, max_depth=20, max_features=log2, min_samples_split=2, n_estimators=500, subsample=0.5; total time=   3.6s
[CV] END criterion=friedman_mse, learning_rate=0.01, loss=squared_error, max_depth=20, max_features=log2, min_samples_split=2, n_estimators=500, subsample=0.5; total time=   3.6s
[CV] END criterion=friedman_mse, learning_rate=0.01, loss=squared_error, max_depth=20, max_features=log2, min_samples_split=2, n_estimators=500, subsample=0.5; total time=   3.7s
[CV] END criterion=squared_error, learning_rate=0.1, loss=quantile, max_depth=10, max_features=sqrt, min_samples_split=2, n_estimators=400, subsample=1.0; total time=   4.9s
[CV] END criterion=squared_error, learning_rate=0.1, loss=quantile, max_depth=10, max_features=sqrt, min_samples_split=2, n_estimators=400, subsample=1.0; total time=   5.2s
[CV] END criterion=squared_error, learning_rate=0.1, loss=quantile, max_depth=10, max_features=sqrt, min_samples_sp

In [34]:
models = {
    "Gradient Boost Regressor": GradientBoostingRegressor(subsample=1.0, n_estimators=500, min_samples_split=2, max_features='sqrt', max_depth=5, loss='squared_error', learning_rate=0.1, criterion='squared_error')
    
}
for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_mse, model_train_rmse, model_train_r2 = model_evaluation(y_train, y_train_pred)

    model_test_mae , model_test_mse, model_test_rmse, model_test_r2 = model_evaluation(y_test, y_test_pred)

    
    print(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- Mean Squared Error: {:.4f}".format(model_train_mse))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- Mean Squared Error: {:.4f}".format(model_test_mse))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    
    print('='*35)
    print('\n')

Gradient Boost Regressor
Model performance for Training set
- Root Mean Squared Error: 96498.2682
- Mean Absolute Error: 66133.2292
- Mean Squared Error: 9311915770.3729
- R2 Score: 0.9886
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 212113.5678
- Mean Absolute Error: 95621.4014
- Mean Squared Error: 44992165639.3461
- R2 Score: 0.9402


